# Examples of Plotting FV3 Forecast Files

FV3 forecasts are written out in two separate files at each output time. Files prefixed `dynf` contain 3D dynamics variables, while files prefixed `phyf` contain physics forecast information, mainly at the surface.

This first cell is to run the common blocks before calling plotting, with input for the forecast hours - fhr. This way the whole notebook looks cleaner.

Main functions are to import packages, read in and define data, define plotting functions. 

In [ ]:
import math
import os
import sys
from argparse import Namespace
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, shiftgrid
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LinearSegmentedColormap
from datetime import date, datetime, timedelta
import glob
import metpy.calc as mpcalc
import metpy
from metpy.plots import Hodograph, SkewT
from metpy.units import pandas_dataframe_to_unit_arrays, units
# plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
# # Add units to the data arrays
# p = p * units.mbar
# T = T * units.degC
# Td = Td * units.degC
# spd = spd * units.knot
# direc = direc * units.deg

In [ ]:
station='SLC'

In [ ]:
# initdate='2019043018'
# fhr=6
# stn={'LZK':(34.84, -92.26), 'SGF':(37.23, -93.38), 'OUN':(35.18, -97.44), 'FWD':(32.83, -97.3)}


In [ ]:
initdate='2017121306'
fhr=6
stn={'SLC':(40.77, -111.95), 'LKN':(40.86, -115.73)}


In [ ]:
# get station lat/lon
lat_obs, long_obs=stn[station]
if long_obs < 0:
    long_obs = 360 + long_obs 
print(lat_obs, long_obs)

In [ ]:
inithr=initdate[8:10]

In [ ]:
def get_validdate(initdate, fhr):
    validdate=(datetime.strptime(initdate,"%Y%m%d%H")+timedelta(hours=fhr)).strftime("%Y%m%d%H")
    print(validdate)
    validday=validdate[0:8]
    print(validday)
    validhr=validdate[8:10]
    print(validhr)
    return validdate, validday, validhr
    
validdate, validday, validhr=get_validdate(initdate, fhr)

In [ ]:
# Provide a file path to a forecast directory. 
# The example below creates a dictionary containing 2 experiments, expt, through the first 4 forecast hours (including 0)

exptlist=['k2n3.no_cu_gf', 'k6n2.no_cu_gf.L82_2mb']
expt=exptlist[1]
# file_path = '/scratch1/BMC/wrfruc/chunhua/fv3sar-testing/code/FV3SAR-DA/expt_dirs/expt_convection-tests/GSD_HRRR3km.GSD.dt30.{expt}/'+f'{initdate}'
# # files = {expt: {x: [os.path.join(file_path.format(expt=expt), x + f'{i:03d}.nc') for i in range(last_fhr + 1)]  for x in ['dynf', 'phyf']} for expt in ['delt_max0.002.dt100.k7n5', 'delt_max0.008.dt100.k7n5']}
# files = {expt: {x: [os.path.join(file_path.format(expt=expt), x + f'{i:03d}.nc') for i in range(last_fhr + 1)]  for x in ['dynf', 'phyf']} for expt in exptlist }


In [ ]:
file_path = f'/scratch1/BMC/wrfruc/chunhua/fv3sar-testing/code/FV3SAR-DA/expt_dirs/expt_convection-tests/GSD_HRRR3km.GSD.dt30.{expt}/{initdate}'
dynfilenc = os.path.join(file_path, f'dynf{fhr:03d}.nc')
phyfilenc = os.path.join(file_path, f'phyf{fhr:03d}.nc')
# print(f'dynf{fhr:03d}.nc')
akbkfilenc = os.path.join(file_path, 'atmos_static.nc')

In [ ]:
dynfile = Dataset(dynfilenc,'r')
phyfile = Dataset(phyfilenc,'r')
akbkfile = Dataset(akbkfilenc,'r')

In [ ]:
akbkfile = Dataset(akbkfilenc, 'r')
for v, info in akbkfile.variables.items():
    print(v, ':', info.long_name, info.units, info.shape)

In [ ]:
dynfile = Dataset(dynfilenc,'r')
for v, info in dynfile.variables.items():
    print(v, ':', info.long_name, info.units, info.shape)

In [ ]:
t3d=np.squeeze(dynfile['tmp'][::])
u3d=np.squeeze(dynfile['ugrd'][::])
v3d=np.squeeze(dynfile['vgrd'][::])
psfc=np.squeeze(dynfile['pressfc'][::])
spfh3d=np.squeeze(dynfile['spfh'][::])

print(np.shape(t3d))
print('T:', np.amin(t3d), np.amax(t3d))
print(np.shape(u3d))
print(np.shape(v3d))
print(np.shape(spfh3d))
print(np.shape(psfc))

ny=np.shape(psfc)[0]
nx=np.shape(psfc)[1]
print(ny)
print(nx)

In [ ]:
ak=np.squeeze(akbkfile['pk'][::])
bk=np.squeeze(akbkfile['bk'][::])


### This cell calculates pressure (phalf) based on ak + bk * Psfc

In [ ]:
nlev=np.shape(ak)[0]
pres = np.array([ak[i] + bk[i] * psfc for i in range(nlev)]) 
print(np.shape(pres))
# print(pres[:,5,5])

### This cell calculates pfull from phalf

In [ ]:
print(np.shape(pres))
presf = (pres[1:,:,:] + pres[0:-1, :,: ])/2
print(np.shape(presf))
print(presf[:,5,5])

In [ ]:
lat = np.squeeze(dynfile['grid_yt'][::]) * 180 / math.pi
lon = np.squeeze(dynfile['grid_xt'][::]) * 180 / math.pi
print(np.shape(lat))
print(np.amax(lat), np.amin(lat))
print(np.shape(lon))
print(np.amax(lon), np.amin(lon))


### This cell serves the important role of finding index of lat/lon in the dimensions

In [ ]:
y, x = np.unravel_index((np.abs(lat - lat_obs) + np.abs(lon - long_obs)).argmin(), lat.shape)
print(y,x)
print(lat[y,x], lon[y,x])

### This cell get the 1-D profile of T, p, u, v, spfh at obs location

In [ ]:
T=t3d[:, y, x] - 273.15
# print(T)
T=T*units.degC
p=presf[:, y, x]/100 * units.hPa
# print(p)
u=u3d[:, y, x]*1.94384 * units.knots
v=v3d[:, y, x]*1.94384 * units.knots
spfh=spfh3d[:, y, x] * units.kg/units.kg
# print(u)
# print(v)

### Calculates dewpoint temperature.
metpy.calc.dewpoint_from_specific_humidity(pressure, temperature, specific_humidity)
https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.dewpoint_from_specific_humidity.html


In [ ]:
Td=mpcalc.dewpoint_from_specific_humidity(pressure=p,temperature=T,specific_humidity=spfh)

In [ ]:
def plot_skewT(p, T, Td, u, v):
    # Change default to be better for skew-T
    fig = plt.figure(figsize=(9, 11))

    # Initiate the skew-T plot type from MetPy class loaded earlier
    skew = SkewT(fig, rotation=45)

    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p, T, 'r')
    skew.plot(p, Td, 'g')
#     skew.plot_barbs(p[::3], u[::3], v[::3], y_clip_radius=0.03)
    skew.plot_barbs(p, u, v, y_clip_radius=0.03)

    # Set some appropriate axes limits for x and y
    skew.ax.set_xlim(-30, 40)
    skew.ax.set_ylim(1020, 100)

    # Add the relevant special lines to plot throughout the figure
    skew.plot_dry_adiabats(t0=np.arange(233, 533, 10) * units.K,
                           alpha=0.25, color='orangered')
    skew.plot_moist_adiabats(t0=np.arange(233, 400, 5) * units.K,
                             alpha=0.25, color='tab:green')
    skew.plot_mixing_lines(p=np.arange(1000, 99, -20) * units.hPa,
                           linestyle='dotted', color='tab:blue')

    # Add some descriptive titles
#     plt.title('{} Sounding'.format(station), loc='left')
    plt.title(f'Sounding at {station}  (lat={lat_obs}, lon={np.around(long_obs-360,3)}) \n')
    plt.title(f'Valid Time: {validdate}     ', loc='right')
    plt.title(expt, loc='left')

    plt.show()

#### The following sections make the plots

In [ ]:
plot_skewT(p, T, Td, u, v)